In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import time
import dash
import openpyxl
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:

    df = pd.read_csv("statusinvest-busca-avancada.csv", sep=';', decimal=',', thousands='.', encoding='ISO-8859-1' )
    
    df['P/L_IDEAL'] = df[' VPA'] / df[' LPA']
    df['UP/DOWNSIDE'] = (df[' VPA'] / df['PRECO'])-1
    
    valuation = df.loc[:, ['TICKER', 'PRECO', ' VPA' , 'DY', ' LPA', 'P/L', 'P/L_IDEAL', 'UP/DOWNSIDE', 'ROE', 
                           'P/VP', 'EV/EBIT' ,'LIQ. CORRENTE', 'MARG. LIQUIDA', 'DIV. LIQ. / PATRI.']]
    valuation.fillna(value=0, inplace=True)
    valuation.rename(columns={'LIQ. CORRENTE': "LIQ.CORR" }, inplace=True)
    valuation.rename(columns={'MARG. LIQUIDA': "MARG.LIQ" }, inplace=True)
    valuation.rename(columns={'DIV. LIQ. / PATRI.': "DIV.LIQ/PL" }, inplace=True)
    
    
    #valuation = valuation.loc[valuation['DY'] >= 10].loc[
    #    valuation['P/L'] >= 0.01].loc[valuation['P/L'] < 15].loc[
    #    valuation['ROE'] >= 10].loc[
    #    valuation['EV/EBIT'] >= 0.01].loc[valuation['EV/EBIT'] <= 15].loc[
    #    valuation['LIQ.CORR'] >= 0.5]


In [3]:
#Instalando o driver
servico = Service(ChromeDriverManager().install())

#Abrindo o navegador
navegador = webdriver.Chrome(service=servico)

In [4]:
ticker = valuation['TICKER']
lista_stocks = ticker.to_list()

lista_indicadores_stocks = []
for stocks in lista_stocks:

    #Criando url
    url = f"https://statusinvest.com.br/acoes/{stocks}"

    
    try:
        #Abrindo o navegador
        navegador.get(url)

        time.sleep(0.25)
    
        #Coletando infos
        PRECO = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[1]/div/div[1]/strong').text
        MIN_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[2]/div/div[2]/div/span[2]').text
        MAX_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[3]/div/div[2]/div/span[2]').text
        MIN_52S = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[2]/div/div[1]/strong').text
        MAX_52S = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[3]/div/div[1]/strong').text
        DY = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[4]/div/div[1]/strong').text
        GAIN_12M = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[5]/div/div[1]/strong').text
        GAIN_MES = navegador.find_element(By.XPATH, '//*[@id="main-2"]/div[2]/div/div[1]/div/div[5]/div/div[2]/div/span[2]/b').text
        QTDEPAPEL = navegador.find_element(By.XPATH, '//*[@id="company-section"]/div[1]/div/div[2]/div[9]/div/div/strong').text
        SEGMENTO = navegador.find_element(By.XPATH, '//*[@id="company-section"]/div[1]/div/div[3]/div/div[3]/div/div/div/a/strong').text
        
        

        #Criando dict
        dicionario = {  "stocks": stocks, 
                        "PRECO": PRECO,
                        "MIN_MES": MIN_MES, 
                        "MAX_MES": MAX_MES, 
                        "MIN_52S": MIN_52S, 
                        "MAX_52S": MAX_52S, 
                        "DY": DY,
                        "VAL_12M": GAIN_12M,
                        "VAL_MES": GAIN_MES,
                        "QTDEPAPEL": QTDEPAPEL,
                        "SEGMENTO": SEGMENTO
                        
                        }

        #Criando lista de dicts
        lista_indicadores_stocks.append(dicionario)
    except Exception as e:
        print(e)

#Fecha o navegador
navegador.quit()


In [5]:
list_stocks = pd.DataFrame.from_dict(lista_indicadores_stocks)

list_stocks = list_stocks.replace('-', '')
list_stocks = list_stocks.replace('-%', '')


list_stocks['MIN_MES'] = (list_stocks['MIN_MES']).str.lstrip('R$ ')
list_stocks['MAX_MES'] = (list_stocks['MAX_MES']).str.lstrip('R$ ')
list_stocks['MIN_MES'] = (list_stocks['MIN_MES']).str.strip('- ')
list_stocks['MAX_MES'] = (list_stocks['MAX_MES']).str.strip('- ')



list_stocks['VAL_12M'] = (list_stocks['VAL_12M']).str.rstrip('%')
list_stocks['VAL_MES'] = (list_stocks['VAL_MES']).str.rstrip('%')

list_stocks.to_csv('list_stocks.csv')



list_stocks = pd.read_csv("list_stocks.csv", sep=',', decimal=',', thousands='.', encoding='ISO-8859-1' )

list_stocks = list_stocks.drop(['Unnamed: 0'],axis=1)
list_stocks.fillna(value=0, inplace=True)


list_stocks = list_stocks.replace('AcessÃ³rios', 'Acessorios')
list_stocks = list_stocks.replace('Armas e MuniÃ§Ãµes', 'Armas e Municoes')
list_stocks = list_stocks.replace('Artefatos de Ferro e AÃ§o', 'Artefatos de Ferro e Aco')
list_stocks = list_stocks.replace('AutomÃ³veis e Motocicletas', 'Automoveis e Motocicletas')
list_stocks = list_stocks.replace('AÃ§ucar e Alcool', 'Acucar e Alcool')
list_stocks = list_stocks.replace('CalÃ§ados', 'Calcados')
list_stocks = list_stocks.replace('ConstruÃ§Ã£o Pesada', 'Construcao Pesada')
list_stocks = list_stocks.replace('EletrodomÃ©sticos', 'Eletrodomesticos')
list_stocks = list_stocks.replace('Energia ElÃ©trica', 'Energia Eletrica')
list_stocks = list_stocks.replace('Equipamentos e ServiÃ§os', 'Equipamentos e Servicos')
list_stocks = list_stocks.replace('ExploraÃ§Ã£o de ImÃ³veis', 'Exploracao de Imoveis')
list_stocks = list_stocks.replace('ExploraÃ§Ã£o de Rodovias', 'Exploracao de Rodovias')
list_stocks = list_stocks.replace('ExploraÃ§Ã£o. Refino e DistribuiÃ§Ã£o', 'Exploracao Refino e Distribuicao')
list_stocks = list_stocks.replace('GestÃ£o de Recursos e Investimentos', 'Gestao Recursos e Investimentos')
list_stocks = list_stocks.replace('GÃ¡s', 'Gas')
list_stocks = list_stocks.replace('IncorporaÃ§Ãµes', 'Incorporacoes')
list_stocks = list_stocks.replace('IntermediaÃ§Ã£o ImobiliÃ¡ria', 'Intermediacao Imobiliaria')
list_stocks = list_stocks.replace('LogÃ­stica', 'Logistica')
list_stocks = list_stocks.replace('Material AeronÃ¡utico e de Defesa', 'Material Aeronautico e Defesa')
list_stocks = list_stocks.replace('Material RodoviÃ¡rio', 'Material Rodoviario')
list_stocks = list_stocks.replace('Minerais MetÃ¡licos', 'Minerais Metalicos')
list_stocks = list_stocks.replace('MÃ¡q. e Equip. ConstruÃ§Ã£o e AgrÃ­colas', 'Maq Equip Construcao e Agricolas')
list_stocks = list_stocks.replace('MÃ¡q. e Equip. Industriais', 'Maq Equip Industriais')
list_stocks = list_stocks.replace('MÃ³veis', 'Moveis')
list_stocks = list_stocks.replace('PetroquÃ­micos', 'Petroquimicos')
list_stocks = list_stocks.replace('Produtos para ConstruÃ§Ã£o', 'Produtos para Construcao')
list_stocks = list_stocks.replace('ProduÃ§Ã£o de Eventos e Shows', 'Producao de Eventos e Shows')
list_stocks = list_stocks.replace('ProduÃ§Ã£o e DifusÃ£o de Filmes e Programas', 'Producao Difusao de Filmes e Programas')
list_stocks = list_stocks.replace('Programas de FidelizaÃ§Ã£o', 'Programas de Fidelizacao')
list_stocks = list_stocks.replace('Programas e ServiÃ§os', 'Programas e Servicos')
list_stocks = list_stocks.replace('QuÃ­micos Diversos', 'Quimicos Diversos')
list_stocks = list_stocks.replace('Serv.MÃ©d.Hospit..AnÃ¡lises e DiagnÃ³sticos', 'Serv Mao Hosp Analises e Diagnosticos')
list_stocks = list_stocks.replace('ServiÃ§os Diversos', 'Servicos Diversos')
list_stocks = list_stocks.replace('ServiÃ§os Educacionais', 'Servicos Educacionais')
list_stocks = list_stocks.replace('ServiÃ§os Financeiros Diversos', 'Servicos Financeiros Diversos')
list_stocks = list_stocks.replace('Soc. CrÃ©dito e Financiamento', 'Soc Credito e Financiamento')
list_stocks = list_stocks.replace('Tecidos. VestuÃ¡rio e CalÃ§ados', 'Tecidos Vestuario e Calcados')
list_stocks = list_stocks.replace('TelecomunicaÃ§Ãµes', 'Telecomunicacoes')
list_stocks = list_stocks.replace('Transporte AÃ©reo', 'Transporte Aereo')
list_stocks = list_stocks.replace('Transporte FerroviÃ¡rio', 'Transporte Ferroviario')
list_stocks = list_stocks.replace('Transporte HidroviÃ¡rio', 'Transporte Hidroviario')
list_stocks = list_stocks.replace('UtensÃ­lios DomÃ©sticos', 'Utensilios Domesticos')
list_stocks = list_stocks.replace('VestuÃ¡rio', 'Vestuario')
list_stocks = list_stocks.replace('Ãgua e Saneamento', 'Agua e Saneamento')


In [6]:
valuation.rename(columns={'TICKER': "TICKERS" }, inplace=True)
valuation

,TICKERS,PRECO,VPA,DY,LPA,P/L,P/L_IDEAL,UP/DOWNSIDE,ROE,P/VP,EV/EBIT,LIQ.CORR,MARG.LIQ,DIV.LIQ/PL
0,AALR3,7.84,7.32,0.00,-2.42,-3.13,-3.024793,-0.066327,-33.11,1.04,-157.54,0.55,-25.30,1.03
1,ABCB4,19.23,23.98,8.43,3.48,5.35,6.890805,0.247010,14.50,0.78,4.25,1.50,19.35,0.00
2,ABEV3,13.69,5.40,5.46,0.91,15.39,5.934066,-0.605551,16.78,2.58,12.12,1.01,17.26,-0.10
3,ADHM3,1.56,-1.21,0.00,-0.38,-4.09,3.184211,-1.775641,-31.47,-1.29,-5.69,0.00,-18860.61,0.00
4,AERI3,0.98,1.10,0.00,-0.14,-6.97,-7.857143,0.122449,-12.65,0.88,6.19,1.63,-3.59,1.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,WIZC3,6.29,2.59,7.20,0.66,9.41,3.924242,-0.588235,25.32,2.38,2.72,0.84,10.49,0.10
616,WLMM3,23.11,17.94,3.14,3.11,8.08,5.768489,-0.223713,17.35,1.40,7.22,2.48,5.11,0.00
617,WLMM4,39.00,17.94,2.32,3.11,12.04,5.768489,-0.540000,17.35,2.09,7.22,2.48,5.11,0.00
618,YDUQ3,20.60,10.16,0.00,0.36,56.84,28.222222,-0.506796,3.51,2.00,13.34,1.56,2.27,1.36


In [7]:
list_stocks.rename(columns={'stocks': "TICKERS" }, inplace=True)
list_stocks

,TICKERS,PRECO,MIN_MES,MAX_MES,MIN_52S,MAX_52S,DY,VAL_12M,VAL_MES,QTDEPAPEL,SEGMENTO
0,AALR3,7.84,7.55,7.93,7.55,24.30,0.00,-60.94,0.51,118292816,Serv Mao Hosp Analises e Diagnosticos
1,ABCB4,19.23,18.35,19.23,15.27,21.24,8.43,2.72,3.00,226090118,Bancos
2,ABEV3,13.69,13.69,14.01,12.86,16.03,5.46,-7.00,-1.30,15753833284,Cervejas e Refrigerantes
3,ADHM3,1.56,1.56,2.10,0.00,0.00,0.00,0.00,0.00,16307545,Serv Mao Hosp Analises e Diagnosticos
4,AERI3,0.98,0.94,1.03,0.88,2.38,0.00,-58.82,1.03,766213456,Maq Equip Industriais
...,...,...,...,...,...,...,...,...,...,...,...
615,WIZC3,6.29,6.13,6.46,4.91,8.41,7.20,-21.86,0.80,159907282,Corretoras de Seguros
616,WLMM3,23.11,23.11,25.17,19.22,25.17,3.14,20.24,-8.18,36414670,Material de Transporte
617,WLMM4,39.00,37.49,39.00,14.88,39.00,2.32,45.25,2.39,36414670,Material de Transporte
618,YDUQ3,20.60,19.87,21.11,6.47,23.70,0.00,80.54,-2.42,309088851,Servicos Educacionais


In [8]:
investing = pd.merge(valuation, list_stocks, on='TICKERS')
investing

,TICKERS,PRECO_x,VPA,DY_x,LPA,P/L,P/L_IDEAL,UP/DOWNSIDE,ROE,P/VP,...,PRECO_y,MIN_MES,MAX_MES,MIN_52S,MAX_52S,DY_y,VAL_12M,VAL_MES,QTDEPAPEL,SEGMENTO
0,AALR3,7.84,7.32,0.00,-2.42,-3.13,-3.024793,-0.066327,-33.11,1.04,...,7.84,7.55,7.93,7.55,24.30,0.00,-60.94,0.51,118292816,Serv Mao Hosp Analises e Diagnosticos
1,ABCB4,19.23,23.98,8.43,3.48,5.35,6.890805,0.247010,14.50,0.78,...,19.23,18.35,19.23,15.27,21.24,8.43,2.72,3.00,226090118,Bancos
2,ABEV3,13.69,5.40,5.46,0.91,15.39,5.934066,-0.605551,16.78,2.58,...,13.69,13.69,14.01,12.86,16.03,5.46,-7.00,-1.30,15753833284,Cervejas e Refrigerantes
3,ADHM3,1.56,-1.21,0.00,-0.38,-4.09,3.184211,-1.775641,-31.47,-1.29,...,1.56,1.56,2.10,0.00,0.00,0.00,0.00,0.00,16307545,Serv Mao Hosp Analises e Diagnosticos
4,AERI3,0.98,1.10,0.00,-0.14,-6.97,-7.857143,0.122449,-12.65,0.88,...,0.98,0.94,1.03,0.88,2.38,0.00,-58.82,1.03,766213456,Maq Equip Industriais
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,WIZC3,6.29,2.59,7.20,0.66,9.41,3.924242,-0.588235,25.32,2.38,...,6.29,6.13,6.46,4.91,8.41,7.20,-21.86,0.80,159907282,Corretoras de Seguros
616,WLMM3,23.11,17.94,3.14,3.11,8.08,5.768489,-0.223713,17.35,1.40,...,23.11,23.11,25.17,19.22,25.17,3.14,20.24,-8.18,36414670,Material de Transporte
617,WLMM4,39.00,17.94,2.32,3.11,12.04,5.768489,-0.540000,17.35,2.09,...,39.00,37.49,39.00,14.88,39.00,2.32,45.25,2.39,36414670,Material de Transporte
618,YDUQ3,20.60,10.16,0.00,0.36,56.84,28.222222,-0.506796,3.51,2.00,...,20.60,19.87,21.11,6.47,23.70,0.00,80.54,-2.42,309088851,Servicos Educacionais


In [9]:
investing.sort_index(axis=1, inplace=True)
investing

#temperatures.columns = sorted(temperatures.columns)
#investing = investing[['TICKERS', 'PRECO_x']]

,LPA,VPA,DIV.LIQ/PL,DY_x,DY_y,EV/EBIT,LIQ.CORR,MARG.LIQ,MAX_52S,MAX_MES,...,P/VP,PRECO_x,PRECO_y,QTDEPAPEL,ROE,SEGMENTO,TICKERS,UP/DOWNSIDE,VAL_12M,VAL_MES
0,-2.42,7.32,1.03,0.00,0.00,-157.54,0.55,-25.30,24.30,7.93,...,1.04,7.84,7.84,118292816,-33.11,Serv Mao Hosp Analises e Diagnosticos,AALR3,-0.066327,-60.94,0.51
1,3.48,23.98,0.00,8.43,8.43,4.25,1.50,19.35,21.24,19.23,...,0.78,19.23,19.23,226090118,14.50,Bancos,ABCB4,0.247010,2.72,3.00
2,0.91,5.40,-0.10,5.46,5.46,12.12,1.01,17.26,16.03,14.01,...,2.58,13.69,13.69,15753833284,16.78,Cervejas e Refrigerantes,ABEV3,-0.605551,-7.00,-1.30
3,-0.38,-1.21,0.00,0.00,0.00,-5.69,0.00,-18860.61,0.00,2.10,...,-1.29,1.56,1.56,16307545,-31.47,Serv Mao Hosp Analises e Diagnosticos,ADHM3,-1.775641,0.00,0.00
4,-0.14,1.10,1.11,0.00,0.00,6.19,1.63,-3.59,2.38,1.03,...,0.88,0.98,0.98,766213456,-12.65,Maq Equip Industriais,AERI3,0.122449,-58.82,1.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,0.66,2.59,0.10,7.20,7.20,2.72,0.84,10.49,8.41,6.46,...,2.38,6.29,6.29,159907282,25.32,Corretoras de Seguros,WIZC3,-0.588235,-21.86,0.80
616,3.11,17.94,0.00,3.14,3.14,7.22,2.48,5.11,25.17,25.17,...,1.40,23.11,23.11,36414670,17.35,Material de Transporte,WLMM3,-0.223713,20.24,-8.18
617,3.11,17.94,0.00,2.32,2.32,7.22,2.48,5.11,39.00,39.00,...,2.09,39.00,39.00,36414670,17.35,Material de Transporte,WLMM4,-0.540000,45.25,2.39
618,0.36,10.16,1.36,0.00,0.00,13.34,1.56,2.27,23.70,21.11,...,2.00,20.60,20.60,309088851,3.51,Servicos Educacionais,YDUQ3,-0.506796,80.54,-2.42
